In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import aesara
import aesara.tensor as aet

lpmc = pd.read_csv("../data/lpmc.dat", sep="\t")
display(lpmc.index.name)
lpmc["travel_mode"] = lpmc["travel_mode"].astype("int64")
display(lpmc["travel_mode"].dtype)
lpmc = lpmc.sample(frac=1.0).reset_index(drop=True)
lpmc.index.name = "index"
lpmc

None

dtype('int64')

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
index,,,,,,,,,,,,,,,,,,,,,
0,17461,3686,2,0,3,1,6,1,0.0,1,...,0.280278,0.333333,0.150000,0.100000,1,0.868889,2.90,2.13,10.5,0.740090
1,45081,9632,0,0,3,4,6,1,1.0,2,...,0.085000,0.000000,0.224722,0.000000,0,0.163611,1.50,0.53,0.0,0.200340
2,60906,13054,0,0,4,3,1,1,1.0,3,...,0.198889,0.233333,0.000000,0.133333,1,0.177500,1.70,0.70,0.0,0.255086
3,1606,335,2,0,3,3,6,1,0.5,1,...,0.139167,0.000000,0.316667,0.000000,0,0.189722,0.75,0.48,0.0,0.225476
4,80039,17368,4,0,3,2,1,3,0.0,3,...,0.114167,0.000000,0.837500,0.098611,1,0.396389,0.00,1.34,0.0,0.489839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81081,75154,16251,0,1,1,5,6,1,1.0,3,...,0.051111,0.000000,0.072500,0.000000,0,0.025556,1.50,0.06,0.0,0.032609
81082,9073,1883,0,0,3,4,1,5,0.0,1,...,0.234444,0.350000,0.000000,0.100000,1,0.737500,0.00,1.75,10.5,0.589454
81083,37109,7916,0,1,3,3,1,5,0.0,2,...,0.166667,0.000000,0.322778,0.000000,0,0.293056,0.00,0.89,0.0,0.229384


In [2]:
choice = "travel_mode"
alts = {
    1: "walk",
    2: "cycle",
    3: "pt",
    4: "drive",
}

mk = min(alts.keys())
alts = {key-mk: val for key, val in alts.items()}
lpmc[choice] = lpmc[choice] - mk

ds = xr.Dataset({name: lpmc[name] for name in lpmc.columns})
ds.attrs["alts"] = alts
ds.attrs["choice"] = choice
len(ds.index)

81086

In [12]:
for name in ds:
    if name == choice:
        ds[name].attrs["y"] = aet.ivector(name)
    else:
        ds[name].attrs["x"] = aet.vector(name)

ds.attrs["x"] = [ds[name].x for name in ds if "x" in ds[name].attrs]
ds.attrs["y"] = ds[choice].y
ds.attrs["all"] = ds.x + [ds.y]
ds

<xarray.Dataset>
Dimensions:                  (index: 81086)
Coordinates:
  * index                    (index) int64 0 1 2 3 4 ... 81082 81083 81084 81085
Data variables: (12/32)
    trip_id                  (index) int64 17461 45081 60906 ... 10246 37812
    household_id             (index) int64 3686 9632 13054 ... 7916 2135 8076
    person_n                 (index) int64 2 0 0 2 4 0 0 0 4 ... 1 0 0 0 0 0 3 0
    trip_n                   (index) int64 0 0 0 0 0 0 0 3 0 ... 1 4 2 1 0 1 1 2
    travel_mode              (index) int64 2 2 3 2 2 2 3 0 0 ... 3 3 0 0 2 2 3 3
    purpose                  (index) int64 1 4 3 3 2 3 2 2 3 ... 5 1 3 5 4 3 5 4
    ...                       ...
    pt_interchanges          (index) int64 1 0 1 0 1 0 0 0 0 ... 0 0 0 0 1 0 1 1
    dur_driving              (index) float64 0.8689 0.1636 ... 0.3075 0.3447
    cost_transit             (index) float64 2.9 1.5 1.7 0.75 ... 0.0 0.0 3.79
    cost_driving_fuel        (index) float64 2.13 0.53 0.7 ... 0.89 1.1 1.22
    cost_driving_ccharge     (index) float64 10.5 0.0 0.0 0.0 ... 0.0 0.0 0.0
    driving_traffic_percent  (index) float64 0.7401 0.2003 ... 0.1147 0.5858
Attributes:
    alts:     {0: 'walk', 1: 'cycle', 2: 'pt', 3: 'drive'}
    choice:   travel_mode
    x:        [trip_id, household_id, person_n, trip_n, purpose, fueltype, fa...
    y:        travel_mode
    all:      [trip_id, household_id, person_n, trip_n, purpose, fueltype, fa...

In [16]:
isinstance(ds["dur_cycling"], xr.DataArray)
any(hasattr(ds["dur_cycling"], value) for value in ["x", "y"])

True

In [398]:
n=round(len(ds.index)*0.8)
ds.attrs["train_index"] = ds.index[:n].values
ds.attrs["valid_index"] = ds.index[n:].values
ds.attrs["scale"] = {var: 1.0 for var in list(ds)}
ds

<xarray.Dataset>
Dimensions:                  (index: 81086)
Coordinates:
  * index                    (index) int64 0 1 2 3 4 ... 81082 81083 81084 81085
Data variables: (12/32)
    trip_id                  (index) int64 68985 56439 19641 ... 6229 70388
    household_id             (index) int64 14912 12113 4160 ... 8280 1274 15231
    person_n                 (index) int64 3 0 2 2 0 0 0 0 2 ... 2 0 0 1 0 0 3 1
    trip_n                   (index) int64 0 1 0 1 0 2 4 7 2 ... 0 2 0 0 1 3 0 3
    travel_mode              (index) int64 2 3 2 0 2 2 3 3 3 ... 2 0 3 2 2 1 2 3
    purpose                  (index) int64 2 2 2 3 4 5 4 3 3 ... 2 3 3 3 3 3 3 3
    ...                       ...
    pt_interchanges          (index) int64 0 0 0 0 1 0 1 0 0 ... 0 0 0 1 1 0 0 1
    dur_driving              (index) float64 0.2425 0.085 ... 0.1547 0.2089
    cost_transit             (index) float64 2.9 1.5 0.0 0.0 ... 3.0 1.5 0.0 3.0
    cost_driving_fuel        (index) float64 0.53 0.23 0.42 ... 0.32 0.39 0.75
    cost_driving_ccharge     (index) float64 10.5 0.0 0.0 0.0 ... 0.0 0.0 0.0
    driving_traffic_percent  (index) float64 0.3986 0.183 ... 0.2765 0.3165
Attributes:
    alts:         {0: 'walk', 1: 'cycle', 2: 'pt', 3: 'drive'}
    choice:       travel_mode
    x:            [trip_id, household_id, person_n, trip_n, purpose, fueltype...
    y:            travel_mode
    all:          [trip_id, household_id, person_n, trip_n, purpose, fueltype...
    train_index:  [    0     1     2 ... 64866 64867 64868]
    valid_index:  [64869 64870 64871 ... 81083 81084 81085]
    scale:        {'trip_id': 1.0, 'household_id': 1.0, 'person_n': 1.0, 'tri...

In [345]:
variables = ["day_of_week", "start_time", "age"]
i = ds.train_index[4:10]
tds = ds.sel(index=i)[variables]
[tds[t].values for t in tds]

[array([5, 4, 2, 6, 7, 4], dtype=int64),
 array([ 8.33333333, 16.75      , 15.66666667, 21.83333333, 16.6       ,
        15.25      ]),
 array([44, 57, 48, 47, 28, 18], dtype=int64)]

In [3]:
import pycmtensor as cmt
import xarray
xarray.__version__

'2023.6.0'